In [3]:
from ultralytics import YOLO
from ultralytics.data.converter import convert_coco
from YOLO_Preprocess import process_fits_and_save_png
import torch
import shutil
import os
import json
import yaml

# Preprocess Data

In [4]:

#create train/test/eval paths for data
# data_path = "/app/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini"
data_path = "/mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini"
yolo_data_path = data_path+"/yolo_annotations"
train_path = data_path+"/yolo_annotations/train"
test_path = data_path+"/yolo_annotations/test"
eval_path = data_path+"/yolo_annotations/eval"
model_path = data_path+"/yolo_annotations/models_and_results"
if not os.path.exists(yolo_data_path): os.mkdir(yolo_data_path)
if not os.path.exists(train_path): os.mkdir(train_path)
if not os.path.exists(test_path): os.mkdir(test_path)
if not os.path.exists(eval_path): os.mkdir(eval_path)
if not os.path.exists(model_path): os.mkdir(model_path)

# Write data.yaml
data = {"path": data_path,
    "train": train_path ,
    "val": train_path , 
    "test": train_path ,
    "nc": 1 ,
    "names": ["Satellite"]}
with open(data_path +"/data.yaml", "w") as file:
    yaml.dump(data, file, default_flow_style=False)    

old_annotations = data_path+"/annotations"
old_images = data_path+"/images"
temp_path = data_path+"/temp"

#convert to coco
convert_coco(labels_dir=old_annotations, save_dir=temp_path)
process_fits_and_save_png(old_images,train_path+"/images")

# Move all files from incorrect annotations folder into labels folder
source_dir = temp_path+"/labels/annotations"
destination_dir = train_path+"/labels"
if not os.path.exists(destination_dir): os.mkdir(destination_dir)
for file_name in os.listdir(source_dir):
    source_file = os.path.join(source_dir, file_name)
    destination_file = os.path.join(destination_dir, file_name)
    if os.path.isfile(source_file):  # Ensure it's a file, not a directory
        shutil.move(source_file, destination_file)
shutil.rmtree(temp_path)


Annotations /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/annotations/annotations.json: 100%|██████████| 100/100 [00:00<00:00, 113.14it/s]


COCO data converted successfully.
Results saved to /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/temp
0/100
10/100
20/100
30/100
40/100
50/100
60/100
70/100
80/100
90/100


# Model Training and Evaluation

In [6]:
model = YOLO("yolo11n.pt").to('cuda')
# model = YOLO("/app/data/models/model.pt").to('cuda')

device = next(model.model.parameters()).device

print(f"Model is using: {model.device}")
print("Cuda Available:", torch.cuda.is_available())

Model is using: cuda:0
Cuda Available: True


### Training

In [11]:
model.train(data=data_path+"/data.yaml", epochs=1,imgsz=1200,batch=0.7)
model.save(model_path+"/model.pt")

New https://pypi.org/project/ultralytics/8.3.86 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/data.yaml, epochs=1, time=None, patience=100, batch=0.7, imgsz=1200, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/yolo_annotations/train/labels.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]Scanning /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/yolo_annotations/train/labels.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]


AutoBatch: Computing optimal batch size for imgsz=1216 at 70.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU) 8.00G total, 6.32G reserved, 4.54G allocated, -2.86G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2590035       23.25         1.046         74.36         33.04      (1, 3, 1216, 1216)                    list
     2590035        46.5         1.659          54.6         53.37      (2, 3, 1216, 1216)                    list
     2590035          93         2.718         80.61         85.79      (4, 3, 1216, 1216)                    list
     2590035         186         5.149         94.11         117.6      (8, 3, 1216, 1216)                    list
     2590035         372         9.347         212.3          1061     (16, 3, 1216, 1216)                    list


/home/david-chaparro/miniconda3/envs/MLTorch/lib/python3.12/site-packages/ultralytics/utils/autobatch.py:90: RuntimeWarning: invalid value encountered in log
  b = int(round(np.exp((np.log(f * fraction) - p[1]) / p[0])))  # y intercept (optimal batch size)


AutoBatch: WARNING ⚠️ error detected: cannot convert float NaN to integer,  using default batch-size 16.


...

                   all        100        100          0          0          0          0

1 epochs completed in 0.010 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 5.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.79 🚀 Python-3.12.2 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.55it/s]


                   all        100        100          0          0          0          0
Speed: 2.0ms preprocess, 6.6ms inference, 0.0ms loss, 5.9ms postprocess per image
Results saved to runs/detect/train3


	.dvc/.gitignore, YOLO_model/model.ipynb, YOLO_model/yolo11n.pt


MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


### Evaluation

In [10]:
# path = eval_path+"/images"
path = train_path+"/images"
model= model.eval()
temp_results = model.predict(path)
results = {}
for item in temp_results:
    filename = item.path.split("/")[-1]
    results[filename] = item.to_df().to_dict("list")
with open(model_path+"/outputs.json", "w") as f:
    json.dump(results, f)


image 1/100 /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/yolo_annotations/train/images/24876_be96bc2f-933d-4c45-ae60-9546625f5fa9_0.png: 448x640 (no detections), 760.0ms
image 2/100 /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/yolo_annotations/train/images/24876_be96bc2f-933d-4c45-ae60-9546625f5fa9_1.png: 448x640 (no detections), 604.0ms
image 3/100 /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/yolo_annotations/train/images/24876_be96bc2f-933d-4c45-ae60-9546625f5fa9_10.png: 448x640 (no detections), 717.1ms
image 4/100 /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/yolo_annotations/train/images/24876_be96bc2f-933d-4c45-ae60-9546625f5fa9_2.png: 448x640 (no detections), 576.8ms
image 5/100 /mnt/c/users/david.chaparro/Documents/data/fits_with_sp3_ra_dec_v3_COCO_dataset_mini/yolo_annotations/train/images/24876_be96bc2f-933d-4c45-ae60-9